## 2. Collect Evidence

In the second phase of SDMT, we collect _evidence_ to attest to the fact that the model realized the properties specified in the previous phase.

We define and instantiate `Measurement`s to generate this evidence. Each individual piece of evidence is a `Value`. Once `Value`s are produced, we can persist them to an _artifact store_ to maintain our evidence across sessions. 

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [ ]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(store_path, exist_ok=True)   # Ensure we are creating the folder if it is not there.

set_context("ns", "OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

Define different folders that will be used as input or output for the data gathering process.

In [ ]:
from pathlib import Path

# The path at which datasets are stored
DATASETS_DIR = Path.cwd() / "data"

# Path where the model files are stored.
MODELS_DIR = Path.cwd() / "model"

# The path at which media is stored
MEDIA_DIR = Path.cwd() / "media"
os.makedirs(MEDIA_DIR, exist_ok=True)

In the next sections, we will define additional functions and gather evidence for the different QA scenarios.

### Fairnesss QAS Measurements

Evidence collected in this section checks for the Fairness scenario defined in the previous step. Note that some functions will be loaded from exernal Python files.

In [ ]:
# General functions.

import garden
import numpy as np

def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder)
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns = {'label':'Label'}, inplace = True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all


def split_data(df_info, df_all):
    """Splits the data into 3 different populations to evaluate them."""
    df_gardenpop = df_info.copy()
    df_gardenpop['Population1'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop['Population2'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop['Population3'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop

    #build populations from test data set that match the garden compositions
    from random import choices

    #build 3 gardens with populations of 1000.
    pop_names = ['Population1', 'Population2', 'Population3']
    gardenpops = np.zeros( (3,1000), int)
    gardenmems = np.zeros( (3,1000), int)

    for j in range(1000):
        for i in range(len(df_gardenpop)):
            my_flower = df_gardenpop.iloc[i]['Common Name']
        
            for g in range(3):
                n_choices = df_gardenpop.iloc[i][pop_names[g]]
                my_choices = df_all[df_all['Common Name'] == my_flower]['model correct'].to_list()
                my_selection = choices(my_choices, k=n_choices)
            
                gardenpops[g][j] += sum(my_selection)
                gardenmems[g][j] += len(my_selection)

    gardenpops

    return gardenpops, gardenmems


def calculate_model_performance_acc(gardenpops, gardenmems):
    """Get accucray of models across the garden populations"""
    gardenacc = np.zeros( (3,1000), float)
    for i in range (1000):
        for g in range(3):
            gardenacc[g][i] = gardenpops[g][i]/gardenmems[g][i]
    gardenacc

    model_performance_acc = []
    for g in range(3):
        avg = round(np.average(gardenacc[g][:]),3)
        std = round(np.std(gardenacc[g][:]),3)
        min = round(np.amin(gardenacc[g][:]),3)
        max = round(np.amax(gardenacc[g][:]),3)
        model_performance_acc.append(round(avg,3))
        
        print("%1d %1.3f %1.3f %1.3f %1.3f" % (g, avg, std, min, max))

    return model_performance_acc


In [ ]:

# Prepare the data.
data = load_data(DATASETS_DIR)
split_data = split_data(data[0], data[1])

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [ ]:
from values.multiple_accuracy import MultipleAccuracy
from mlte.measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the Spec.
accuracy_measurement = ExternalMeasurement("accuracy across gardens", MultipleAccuracy, calculate_model_performance_acc)
accuracy = accuracy_measurement.evaluate(split_data[0], split_data[1])

# Inspect value
print(accuracy)

# Save to artifact store
accuracy.save(force=True)

### Robustness QAS Measurements

Evidence collected in this section checks for the Robustness scenarios.

In [ ]:
# General functions.
import pandas as pd


def calculate_base_accuracy(df_results: pd.DataFrame) -> pd.DataFrame:
    #Calculate the base model accuracy result per data label
    df_pos = df_results[df_results['model correct'] == True].groupby('label').count()
    df_pos.drop(columns = ['prediced_label'], inplace = True)
    df_neg = df_results[df_results['model correct'] == False].groupby('label').count()
    df_neg.drop(columns = ['prediced_label'], inplace = True)
    df_neg.rename(columns = {'model correct':'model incorrect'}, inplace = True)
    df_res = df_pos.merge(df_neg, right_on ='label', left_on = 'label', how = 'outer')
    df_res.fillna(0, inplace = True)
    df_res['model acc'] = df_res['model correct'] / ( df_res['model correct'] + df_res['model incorrect'] )
    df_res['count'] = ( df_res['model correct'] + df_res['model incorrect'] )
    df_res.drop(columns = ['model correct', 'model incorrect'], inplace = True)
    df_res.head()

    return df_res

def calculate_accuracy_per_set(data_folder: str, df_results: pd.DataFrame, df_res: pd.DataFrame) -> pd.DataFrame:
    #Calculate the model accuracy per data label for each blurred data set
    base_filename = 'FlowerModelv1_TestSetResults'
    ext_filename = '.csv'
    set_filename = ['_blur2x8', '_blur5x8', '_blur0x8', '_noR', '_noG','_noB']

    col_root = 'model acc'

    for fs in set_filename:
        filename = os.path.join(data_folder, base_filename + fs + ext_filename)
        colname = col_root + fs
        
        df_temp = pd.read_csv(filename)
        df_temp.drop(columns = ['Unnamed: 0'], inplace = True)
        
        df_pos = df_temp[df_temp['model correct'] == True].groupby('label').count()
        df_pos.drop(columns = ['prediced_label'], inplace = True)
        df_neg = df_results[df_results['model correct'] == False].groupby('label').count()
        df_neg.drop(columns = ['prediced_label'], inplace = True)
        df_neg.rename(columns = {'model correct':'model incorrect'}, inplace = True)
        df_res2 = df_pos.merge(df_neg, right_on ='label', left_on = 'label', how = 'outer')
        df_res2.fillna(0, inplace = True)
        
        df_res2[colname] = df_res2['model correct'] / ( df_res2['model correct'] + df_res2['model incorrect'] )
        df_res2.drop(columns = ['model correct', 'model incorrect'], inplace = True)
        
        df_res = df_res.merge(df_res2, right_on = 'label', left_on = 'label', how = 'outer')

    df_res.head()
    return df_res

def print_model_accuracy(df_res: pd.DataFrame, key: str, name: str):
    model_acc = sum( df_res[key] * df_res['count'] ) / sum( df_res['count'] )
    print(name , model_acc)

In [ ]:
# Prepare all data.
df_results = garden.load_base_results(DATASETS_DIR)
df_res = calculate_base_accuracy(df_results)
df_res = calculate_accuracy_per_set(DATASETS_DIR, df_results, df_res)
df_info = garden.load_taxonomy(DATASETS_DIR)
df_all = garden.merge_taxonomy_with_results(df_res, df_info, "label", "Label")

#fill in missing model accuracy data
df_all['model acc_noR'].fillna(0, inplace = True)
df_all['model acc_noG'].fillna(0, inplace = True)
df_all['model acc_noB'].fillna(0, inplace = True)

Now do the actual measurements. First simply see the model accuracy across blurs.

In [ ]:
#view changes in model accuracy
print_model_accuracy(df_res, 'model acc', 'base model accuracy')
print_model_accuracy(df_res, 'model acc_blur2x8', 'model accuracy with 2x8 blur')
print_model_accuracy(df_res, 'model acc_blur5x8', 'model accuracy with 5x8 blur')
print_model_accuracy(df_res, 'model acc_blur0x8', 'model accuracy with 0x8 blur')

Measure the ranksums (p-value) for all blur cases, using `scipy.stats.ranksums` and the `ExternalMeasurement` wrapper.

In [ ]:
import scipy.stats

from values.ranksums import RankSums
from mlte.measurement import ExternalMeasurement

my_blur = ['2x8', '5x8', '0x8']
for i in range(len(my_blur)):
    # Define measurements.
    ranksum_measurement = ExternalMeasurement(f"ranksums blur{my_blur[i]}", RankSums, scipy.stats.ranksums)

    # Evaluate.
    ranksum: RankSums = ranksum_measurement.evaluate(df_res['model acc'], df_res[f"model acc_blur{my_blur[i]}"])

    # Inspect values
    print(ranksum)

    # Save to artifact store
    ranksum.save(force=True)

Now to next part of the question- is this equal across the phylogenic groups?

First we will check the effect of blur for Clade 2.

In [ ]:
from typing import List

from values.multiple_ranksums import MultipleRanksums

#use the initial result, blur columns to anaylze effect of blur
df_all['delta_2x8'] = df_all['model acc'] - df_all['model acc_blur2x8']
df_all['delta_5x8'] = df_all['model acc'] - df_all['model acc_blur5x8']
df_all['delta_0x8'] = df_all['model acc'] - df_all['model acc_blur0x8']

pops = df_all['Clade2'].unique().tolist()
blurs = ['delta_2x8', 'delta_5x8', 'delta_0x8',]

ranksums: List = []
for i in range(len(blurs)):
    for pop1 in pops:
        for pop2 in pops:
            ranksum_measurement = ExternalMeasurement(f"ranksums clade2 {pop1}-{pop2} blur{blurs[i]}", RankSums, scipy.stats.ranksums)
            ranksum: RankSums = ranksum_measurement.evaluate(df_all[df_all['Clade2'] == pop1 ][blurs[i]], df_all[df_all['Clade2'] == pop2 ][blurs[i]])
            print(f"blur {blurs[i]}: {ranksum}")
            ranksums.append({ranksum.identifier: ranksum.array})

multiple_ranksums_meas = ExternalMeasurement(f"multiple ranksums for clade2", MultipleRanksums, lambda x: x)
multiple_ranksums: MultipleRanksums = multiple_ranksums_meas.evaluate(ranksums)
multiple_ranksums.num_pops = len(pops)
multiple_ranksums.save(force=True)


Now we check between clade 2 and clade 3.

In [ ]:
df_now = df_all[['Clade2', 'Clade 3']].copy().groupby(['Clade2', 'Clade 3']).count().reset_index()
ps1 = df_now['Clade2'].to_list()
ps2 = df_now['Clade 3'].to_list()
print(df_now)

ranksums: List = []
for k in range(len(blurs)):
    print('\n', blurs[k]) 
    for i in range(len(ps1)):
        p1c1 = ps1[i]
        p1c2 = ps2[i]
        for j in range(len(ps1)):
            p2c1 = ps1[j]
            p2c2 = ps2[j]
            if (len(df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p2c2)][blurs[k]])>0 | 
                len(df_all[(df_all['Clade2'] == p2c1) & (df_all['Clade 3'] == p2c2)][blurs[k]])>0):

                ranksum_measurement = ExternalMeasurement(f"ranksums {p1c1}-{p2c2} - {p2c1}-{p2c2} blur{blurs[k]}", RankSums, scipy.stats.ranksums)
                ranksum: RankSums = ranksum_measurement.evaluate(df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p2c2)][blurs[k]],
                                                                 df_all[(df_all['Clade2'] == p2c1) & (df_all['Clade 3'] == p2c2)][blurs[k]])
                ranksums.append({ranksum.identifier: ranksum.array})

multiple_ranksums_meas = ExternalMeasurement(f"multiple ranksums between clade2 and 3", MultipleRanksums, lambda x: x)
multiple_ranksums: MultipleRanksums = multiple_ranksums_meas.evaluate(ranksums)
multiple_ranksums.num_pops = len(ps1)
multiple_ranksums.save(force=True)

### Performance QAS Measurements

Now we collect stored, CPU and memory usage data when predicting with the model, for the Performance scenario.

In [ ]:
# This is the external script that will load and run the model for inference/prediction.
script = Path.cwd() / "model_predict.py"
args = [
    "--images", DATASETS_DIR, 
    "--model", MODELS_DIR / "model_f3_a.json",
    "--weights", MODELS_DIR / "model_f_a.h5"
]

In [ ]:
from mlte.measurement.storage import LocalObjectSize
from mlte.value.types.integer import Integer

store_measurement = LocalObjectSize("model size")
size: Integer = store_measurement.evaluate(MODELS_DIR)
print(size)
size.save(force=True)

In [ ]:
from mlte.measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics

cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
cpu_stats: CPUStatistics = cpu_measurement.evaluate(ProcessMeasurement.start_script(script, args))
print(cpu_stats)
cpu_stats.save(force=True)

In [ ]:
from mlte.measurement.memory import LocalProcessMemoryConsumption, MemoryStatistics

mem_measurement = LocalProcessMemoryConsumption("predicting memory")
mem_stats: MemoryStatistics = mem_measurement.evaluate(ProcessMeasurement.start_script(script, args))
print(mem_stats)
mem_stats.save(force=True)

### Interpretability QAS Measurements.

Now we proceed to gather data about the Interpretability of the model, for the corresponding scenario.

In [ ]:
model_filename = MODELS_DIR / "model_f3_a.json" # The json file of the model to load
weights_filename = MODELS_DIR / "model_f_a.h5" # The weights file for the model 

In [ ]:
from model_analysis import *

# Load the model/
loaded_model = load_model(model_filename, weights_filename)

In [ ]:
# Load and show the image.

flower_img = "sunflower.jpg" # Filename of flower image to use
flower_idx = 42 # Classifier index of associated flower (see OxfordFlower102Labels.csv)

im = read_image(os.path.join(DATASETS_DIR, flower_img))

plt.imshow(im)
plt.axis("off")
plt.show()

In [ ]:
predictions = run_model(im, loaded_model)

baseline, alphas = generate_baseline_and_alphas()

In [ ]:
interpolated_images = interpolate_images(
    baseline=baseline,
    image=im,
    alphas=alphas)

In [ ]:
fig = plt.figure(figsize=(20, 20))

i = 0
for alpha, image in zip(alphas[0::10], interpolated_images[0::10]):
  i += 1
  plt.subplot(1, len(alphas[0::10]), i)
  plt.title(f'alpha: {alpha:.1f}')
  plt.imshow(image)
  plt.axis('off')

plt.tight_layout()

In [ ]:
path_gradients = compute_gradients(
    loaded_model=loaded_model,
    images=interpolated_images,
    target_class_idx=flower_idx)
print(path_gradients.shape)

ig = integral_approximation(
    gradients=path_gradients)
print(ig.shape)

In [ ]:
ig_attributions = integrated_gradients(baseline=baseline,
                                       image=im,
                                       target_class_idx=flower_idx,
                                       loaded_model=loaded_model,
                                       m_steps=240)
print(ig_attributions.shape)

In [ ]:
fig = plot_img_attributions(image=im,
                          baseline=baseline,
                          target_class_idx=flower_idx,
                          loaded_model=loaded_model,
                          m_steps=240,
                          cmap=plt.cm.inferno,
                          overlay_alpha=0.4)

plt.savefig(MEDIA_DIR / "attributions.png")

In [ ]:
from mlte.measurement import ExternalMeasurement
from mlte.value.types.image import Image

# Save to MLTE store.
img_collector = ExternalMeasurement("image attributions", Image)
img = img_collector.ingest(MEDIA_DIR / "attributions.png")
img.save(force=True)